In [ ]:
#imports

import warnings
warnings.filterwarnings('ignore')

from nilmtk import DataSet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import random
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


In [ ]:
def read_data(train_building, meter_key, neg=0):
    #data = pd.HDFStore("/home/soufiane/Bureau/internship/data/ukdale.h5")
    data = DataSet('/home/soufiane/Bureau/internship/data/ukdale.h5')
    data_elec = data.buildings[train_building].elec
    data_meter = data_elec.submeters()[meter_key]

    start_time = data_meter.get_timeframe().start
    #print(start_time.hour)
    #sampling each hour = 3600 seconds
    sampled = next(data_meter.load(sample_period=3600))

    #transform to numpy array of 24*356*4 elements
    M = np.array(sampled)
    lenght = M.shape[0]
    M = M[:-(lenght%24)]
    M = M.reshape(lenght//24,24)

    M = M[~np.all(M == 0, axis=1)]

    M = M.reshape(len(M),24,1)
    for i in range(len(M)):
        M[i] = np.roll(M[i], -start_time.hour)

    #M = preprocessing.normalize(M)
    M = M/(max([max(x) for x in M]))

    U = []
    for x in M:
        RAN = np.array([random.uniform(0, 2) for i in range(24)])
        RAN = np.expand_dims(RAN,axis=2)
        y = x*RAN
        U.append(y)
    V = []
    for x in M:
        RAN = np.array([random.uniform(0, 2) for i in range(24)])
        RAN = np.expand_dims(RAN,axis=2)
        y = x*RAN
        V.append(y)

    for i in range(len(U)):
        U[i] = np.roll(U[i], 8)
        V[i] = np.roll(V[i], -8)

    dada = [0 for i in range(len(M))]
    x_train, x_test, _, __ = train_test_split(M, dada, test_size=0.20)
    u_train, u_test, _, __ = train_test_split(U, dada, test_size=0.20)
    v_train, v_test, _, __ = train_test_split(V, dada, test_size=0.20)

    u_train = np.array(u_train)
    u_test = np.array(u_test)

    if neg==0:
        return x_train, x_test, u_train, u_test
    else:
        return x_train, x_test, u_train, u_test, v_train, v_test